In [ ]:
from analysis.paper import *

In [ ]:
ags = data.load()
#TODO: Push this back into the database
ags = data.with_times(ags)
#upload(plot_hex)
upload(plot_flops_curves, ags)
upload(plot_frontiers, ags)
upload(plot_direct_frontiers, ags)
upload(plot_resid_var, ags)
upload(plot_runtimes, ags)
upload(plot_train_test, ags)
upload(plot_elos)
upload(plot_test, ags)
upload(plot_calibrations)

In [ ]:
overleaf.table(boardsize_hyperparams_table(ags), 'boardsize_hyperparams')